# Dia 2
## Taller de Blockchain



## Criptografia

### HASH

Primero importamos la funcion sha256 de la libreria hashlib. 
Creamos un mensaje cualquiera y lo hasheamos.
Modificamos y hasheamos para ver el cambio, luego volvemos al mensaje original y vemos que el hash no varia.

In [1]:
from hashlib import sha256

mensaje = "Hola, que tal?"
hash = sha256(mensaje.encode()).hexdigest()
print(hash)

0a9b0d922ea756cabe9e6eb43fea5aba1e8500b9d36c37c5482240b83af1379e


In [2]:
mensaje = "Hola, que tal? Bien."
hash = sha256(mensaje.encode()).hexdigest()
print(hash)

4c6b7c49c16ecda271040485b7b5cf5b04e686d6bb01605ced55f18a4b13728f


In [3]:
mensaje = "Hola, que tal?"
hash = sha256(mensaje.encode()).hexdigest()
print(hash)

0a9b0d922ea756cabe9e6eb43fea5aba1e8500b9d36c37c5482240b83af1379e


### Challenge 1 

Crear una funcion que reciba una palabra y te devuelva el Hash de la palabra.
Ejecuta 3 veces la funcion de la siguiente manera:
- La primera con una palabra
- La segunda con un caracter diferente
- La tercera igual que la primera
Imprimir cada palabra con su hash y comparar.



In [2]:
from hashlib import sha256

for i in range(3):
    palabra = input("Ingrese una palabra: ")
    hash = sha256(palabra.encode()).hexdigest()
    print(palabra)
    print(hash)


Hola
e633f4fc79badea1dc5db970cf397c8248bac47cc3acf9915ba60b5d76b0e88f
hola
b221d9dbb083a7f33428d7c2a3c3198ae925614d70210e28716ccaa7cd4ddb79
Hola
e633f4fc79badea1dc5db970cf397c8248bac47cc3acf9915ba60b5d76b0e88f


### Implementacion a la blockchain

In [4]:
from hashlib import sha256
from time import time
import json

class Bloque:
    def __init__(self, index, transacciones,timestamp, hash_anterior):
        self.index = index
        self.transacciones = transacciones
        self.timestamp = timestamp
        self.hash_anterior = hash_anterior

class Blockchain: 
    def __init__(self):
        self.transacciones_pendientes = []
        self.chain = []
        self.crear_bloque_genesis()
    
    # Agregar el hash 0 al bloque genesis
    def crear_bloque_genesis(self):
        genesis_block = Bloque(0, ["Genesis"], time(), "0")
        self.chain.append(genesis_block)
    
    @property
    def last_block(self):
        return self.chain[-1]

    # Creamos la funcion de hasheo que cripte los bloques
    # para poder usar la herramienta debemos convertir todo a strings usando 
    # la herramienta dumps de json, a su vez ordenamos todo.
    @staticmethod
    def criptar(bloque):
        bloque_string = json.dumps(bloque.__dict__, sort_keys=True)
        return sha256(bloque_string.encode()).hexdigest()
    
    def agregar_bloque(self, block):
        self.chain.append(block)
        return True
    
    def agregar_transaccion(self, transaction):
                self.transacciones_pendientes.append(transaction)
    
    # Agregamos el hash del bloque anterior al bloque que se va a cerrar.
    def cerrar_bloque(self):
        if not self.transacciones_pendientes:
            return False
        ultimo_bloque = self.last_block
        nuevo_bloque = Bloque(ultimo_bloque.index + 1, self.transacciones_pendientes, time(), self.criptar(ultimo_bloque))
        self.agregar_bloque(nuevo_bloque)
        self.transacciones_pendientes = []
        return nuevo_bloque

In [12]:
bitcoin = Blockchain()

bitcoin.agregar_transaccion('Primer Mensaje')
bitcoin.agregar_transaccion('Segundo Mensaje')
bitcoin.cerrar_bloque()

print("Bloque 1")
print(bitcoin.chain[1].__dict__)
a = bitcoin.criptar(bitcoin.chain[1])
print("Hash: ")
print(a)

bitcoin.agregar_transaccion('Tercer Mensaje')
bitcoin.cerrar_bloque()
for i in bitcoin.chain:
    print(i.__dict__)

Bloque 1
{'index': 1, 'transacciones': ['Primer Mensaje', 'Segundo Mensaje'], 'timestamp': 1669913631.0535738, 'hash_anterior': 'a0a7fba707c87fcf76fa07355c4c09d129f230551f0240e9cf0edcaee7bb7d68'}
Hash: 
c78329f19b13df30ec64df5fe9b63da368d47c52e55fa4eeb7720ee27590048c
{'index': 0, 'transacciones': ['Genesis'], 'timestamp': 1669913631.05354, 'hash_anterior': '0'}
{'index': 1, 'transacciones': ['Primer Mensaje', 'Segundo Mensaje'], 'timestamp': 1669913631.0535738, 'hash_anterior': 'a0a7fba707c87fcf76fa07355c4c09d129f230551f0240e9cf0edcaee7bb7d68'}
{'index': 2, 'transacciones': ['Tercer Mensaje'], 'timestamp': 1669913631.053685, 'hash_anterior': 'c78329f19b13df30ec64df5fe9b63da368d47c52e55fa4eeb7720ee27590048c'}


## Proof of Work
### Nonce


In [50]:
from hashlib import sha256
from time import time
import json

class Bloque:
    def __init__(self, index, transacciones,timestamp, hash_anterior, nonce=0):
        self.index = index
        
        # Agregamos un nuevo atributo 
        self.nonce = nonce

        self.transacciones = transacciones
        # self.timestamp = timestamp
        self.hash_anterior = hash_anterior

    def criptar(self):
        bloque_string = json.dumps(self.__dict__, sort_keys=True)
        return sha256(bloque_string.encode()).hexdigest()

        

class Blockchain: 
    def __init__(self):
        self.transacciones_pendientes = []
        self.chain = []
        self.crear_bloque_genesis()
    
    def crear_bloque_genesis(self):
        genesis_block = Bloque(0, ["Genesis"], time(), 0)
        genesis_block.hash = genesis_block.criptar()
        self.chain.append(genesis_block)
    
    @property
    def last_block(self):
        return self.chain[-1]
    
    # Agregamos el hash que recibimos de la funcion proof_of_work 
    def agregar_bloque(self, bloque, proof):
        # Almacenamos el hash anterior para verificar la validez
        # previous_hash = self.last_block.hash
        # if previous_hash != bloque.hash_anterior:
        #     return False
        
        # Llamamos a nuestra funcion de validez
        # if not self.es_valido(bloque, proof):
        #     return False
         
        bloque.hash = proof
        self.chain.append(bloque)
        return True
    
    def agregar_transaccion(self, transaction):
                self.transacciones_pendientes.append(transaction)
    # Cambiamos el nombre de la funcion a minar
    def minar(self):
        if not self.transacciones_pendientes:
            return False
        ultimo_bloque = self.last_block
        
        nuevo_bloque = Bloque(ultimo_bloque.index + 1, self.transacciones_pendientes, time(), ultimo_bloque.hash)
        # Agrego la prueba de trabajo a la funcion
        proof = self.proof_of_work(nuevo_bloque)
        
        # Paso el proof para crear el nuevo bloque
        self.agregar_bloque(nuevo_bloque, proof)
        
        self.transacciones_pendientes = []
        return nuevo_bloque


    # Agruegamos la dificultad a nuestra Blockchain
    dificultad = 4
    # Creamos la funcion proof of work
    def proof_of_work(self, bloque):
        # comenzamos con el nonce en 0
        bloque.nonce = 0
        # Criptamos el bloque para ver el hash
        hash = bloque.criptar()
        # Comparamos el hash del bloque ajustado a la dificultad
        while not hash.startswith('0'* self.dificultad):
            # Vamos incrementando el nonce hasta que el hash comience con 
            # la cantidad de ceros en la dificultad
            bloque.nonce += 1

            # Cripamos el bloque con el nuevo valor del nonce
            hash = bloque.criptar()
        return hash

    # Creamos una funcion que devuelva la validez del bloque
    # Verificando si el hash es correcto.
    def es_valido(self, bloque, bloque_hash):
        return (bloque_hash.startswith('0' * Blockchain.difficulty) and
                bloque_hash == bloque.compute_hash())



In [45]:
from hashlib import sha256
import json

class Block:
    def __init__(self, index, transactions, timestamp, previous_hash, nonce=0):
        self.index = index
        self.transactions = transactions
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        self.nonce = nonce

    def compute_hash(self):
        block_string = json.dumps(self.__dict__, sort_keys=True)
        return sha256(block_string.encode()).hexdigest()

import time

class Blockchain: 
    def __init__(self):
        self.unconfirmed_transactions = []
        self.chain = []
        self.create_genesis_block()
 
    def create_genesis_block(self):
        genesis_block = Block(0, [], time.time(), "0")
        genesis_block.hash = genesis_block.compute_hash()
        self.chain.append(genesis_block)
    @property
    def last_block(self):
        return self.chain[-1]
    
    difficulty = 2
    def proof_of_work(self, block):
        block.nonce = 0
        computed_hash = block.compute_hash()
        while not computed_hash.startswith('0' * Blockchain.difficulty):
            block.nonce += 1
            computed_hash = block.compute_hash()
        return computed_hash
    def add_block(self, block, proof):
        previous_hash = self.last_block.hash
        if previous_hash != block.previous_hash:
            return False
        if not self.is_valid_proof(block, proof):
            return False
        block.hash = proof
        self.chain.append(block)
        return True
 
    def is_valid_proof(self, block, block_hash):
        return (block_hash.startswith('0' * Blockchain.difficulty) and
                block_hash == block.compute_hash())

    def add_new_transaction(self, transaction):
            self.unconfirmed_transactions.append(transaction)
 
    def mine(self):
        if not self.unconfirmed_transactions:
            return False
 
        last_block = self.last_block
 
        new_block = Block(index=last_block.index + 1,
                          transactions=self.unconfirmed_transactions,
                          timestamp=time.time(),
                          previous_hash=last_block.hash)
 
        proof = self.proof_of_work(new_block)
        self.add_block(new_block, proof)
        self.unconfirmed_transactions = []
        return new_block.index



In [51]:
blockchain = Blockchain()

blockchain.agregar_transaccion("Hola que tal?")
blockchain.agregar_transaccion("tranquilopa")
blockchain.minar()
blockchain.agregar_transaccion("tranquilopa")
blockchain.minar()
blockchain.agregar_transaccion("tranquilopa")
blockchain.minar()

for i in blockchain.chain:
    print (i.__dict__)



{'index': 0, 'nonce': 0, 'transacciones': ['Genesis'], 'hash_anterior': 0, 'hash': '8dcc0c5c35c74b9502c0e8d948cfa6318f5552733f6b6af6d4b86143fae2ca5e'}
{'index': 1, 'nonce': 45014, 'transacciones': ['Hola que tal?', 'tranquilopa'], 'hash_anterior': '8dcc0c5c35c74b9502c0e8d948cfa6318f5552733f6b6af6d4b86143fae2ca5e', 'hash': '00007bd06fe4d489406d7deb395368a58458b1cda5ea75e15346340a528d97d3'}
{'index': 2, 'nonce': 10407, 'transacciones': ['tranquilopa'], 'hash_anterior': '00007bd06fe4d489406d7deb395368a58458b1cda5ea75e15346340a528d97d3', 'hash': '00009772fbeba9def06dd4eb6e3ab32a2271ddd65ecf297e4a3aaffc78780c15'}
{'index': 3, 'nonce': 101980, 'transacciones': ['tranquilopa'], 'hash_anterior': '00009772fbeba9def06dd4eb6e3ab32a2271ddd65ecf297e4a3aaffc78780c15', 'hash': '000002deb2d2bfaefba891dff25d0fee32fce49b9f7424871a2973d881ef875d'}
